## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np
# Import API key
from config import g_key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv ("Weather_Database/WeatherPy_Database.csv")

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kijang,KR,2021-03-28 01:40:47,35.2442,129.2139,60.80,67,75,10.36,broken clouds
1,1,Kaitangata,NZ,2021-03-28 01:40:47,-46.2817,169.8464,64.00,79,100,11.01,moderate rain
2,2,New Norfolk,AU,2021-03-28 01:40:48,-42.7826,147.0587,64.00,39,75,10.00,broken clouds
10,10,Luderitz,NaN,2021-03-28 01:40:51,-26.6481,15.1594,63.25,68,0,10.76,clear sky
12,12,Bluff,NZ,2021-03-28 01:38:31,-46.6000,168.3333,62.01,84,100,5.01,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                112
City                   112
Country                108
Date                   112
Lat                    112
Lng                    112
Max Temp               112
Humidity               112
Cloudiness             112
Wind Speed             112
Current Description    112
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kijang,KR,60.80,broken clouds,35.2442,129.2139,
1,Kaitangata,NZ,64.00,moderate rain,-46.2817,169.8464,
2,New Norfolk,AU,64.00,broken clouds,-42.7826,147.0587,
12,Bluff,NZ,62.01,light rain,-46.6000,168.3333,
22,Pisco,PE,68.00,clear sky,-13.7000,-76.2167,
26,Castro,BR,64.29,light rain,-24.7911,-50.0119,
27,El Dorado,US,66.00,clear sky,38.7666,-120.5177,
37,Port Alfred,ZA,61.34,light rain,-33.5906,26.8910,
46,Nouadhibou,MR,64.40,clear sky,20.9310,-17.0347,
56,Ponta Do Sol,PT,64.40,broken clouds,32.6667,-17.1000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
#Recall that when trying to parse the data from an API request, we need to use a try-except block to handle the error with 
#a statement and continue the retrieval.

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kijang,KR,60.80,broken clouds,35.2442,129.2139,Samyang Beach Villa
1,Kaitangata,NZ,64.00,moderate rain,-46.2817,169.8464,Kaitangata Motor Camp
2,New Norfolk,AU,64.00,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
12,Bluff,NZ,62.01,light rain,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
22,Pisco,PE,68.00,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
26,Castro,BR,64.29,light rain,-24.7911,-50.0119,CHACARA BAILLY
27,El Dorado,US,66.00,clear sky,38.7666,-120.5177,Bridal Veil Campground
37,Port Alfred,ZA,61.34,light rain,-33.5906,26.8910,The Halyards Hotel
46,Nouadhibou,MR,64.40,clear sky,20.9310,-17.0347,El Medina
56,Ponta Do Sol,PT,64.40,broken clouds,32.6667,-17.1000,Hotel do Campo


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kijang,KR,60.80,broken clouds,35.2442,129.2139,Samyang Beach Villa
1,Kaitangata,NZ,64.00,moderate rain,-46.2817,169.8464,Kaitangata Motor Camp
2,New Norfolk,AU,64.00,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
12,Bluff,NZ,62.01,light rain,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
22,Pisco,PE,68.00,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))